In [ ]:
import pandas as pd 
import numpy as np
df = pd.read_parquet('../scripts/df_merged1.parquet', engine='pyarrow')

In [ ]:
df.shape

In [ ]:
unique_ids = df['objectId'].unique().tolist()

In [ ]:
condition = (df['source'] == 0) | (df['dc_sigflux'] == 0)
df['dc_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
df['nr_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

In [ ]:
df[df['source'] == 0]

In [ ]:
df[['source','dc_weight']].head()

In [ ]:
len(unique_ids)

In [ ]:
import time 
start_time = time.time()



m=0
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
l=1
Q = df[['dc_flux']].iloc[:2*(m+1)]#.values # j:j+m ?

d = {k: 0 for k in unique_ids}
#print(d)

count = 0

for k in unique_ids:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn
        n = len(T)
        zeros = np.zeros(n, dtype=float)
        s_1 = zeros.copy()
        s_2 = zeros.copy()
        f = T['dc_flux'].values
        w = T['dc_weight'].values
        
        for j in range(0,m+1): 
            #f = T['dc_flux'].values[j*2:] # * 2 !!! for r and g right ? 
            #w = T['dc_sigflux_weight'].values[j*2:]
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        s = np.zeros_like(s_d, dtype=float)

        #print(len(s), len(s_n1))
        s[mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
         
            
        #alpha = s[::2] + s[1::2] 
        #print(s)
        alpha = np.repeat(s, 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] # :-j*2 it works but not for 0 

        d[k] = dd[::2] + dd[1::2]
            
            
    if (d[k] <= factor).all():
        print(k, ": ", d[k],"\n")
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
(d['ZTF17aaaeclk']<= factor)

In [ ]:
d['ZTF17aaaeclk']

In [ ]:
Q

In [ ]:
df[df['objectId']=='ZTF17aaaeclk']

In [ ]:
alpha

In [ ]:
1.32646186*np.array((0.000286,0.000636))

#

# with l>0

In [ ]:
import time 
start_time = time.time()



m=1
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
l=1

chunk_size = 2 * (m + 1)
# Calculate the number of chunks
num_chunks = len(df[['dc_flux']]) // chunk_size
# Split the DataFrame into chunks and store them in an array
Q = [df[['dc_flux']].iloc[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]

if m>0: 
    for j in range(1, m+1):
        Q.extend([df[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])

Q = pd.concat(Q) # here we transform Q from list of dataframes to a single data frame  
# we will be able to distinct the values by using l variable s

Number_l = (len(Q)) // (2*(m+1)) 

d = {k: 0 for k in unique_ids[0:10]}
#print(d)

count = 0

for k in unique_ids[0:10]:
    
    T = df[df['objectId'] == k][['dc_flux', 'dc_weight']]
    n = len(T)
    f = T['dc_flux'].values
    w = T['dc_weight'].values
    
    zeros = np.zeros(n, dtype=float)

    for l in range(Number_l):
        #print(l)
        s_1 = zeros.copy()
        s_2 = zeros.copy()
        d[k]=0
        
        for j in range(0,m+1): 
            h = np.tile(Q[2*l*(m+1) +j*2: 2*l*(m+1) + j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        s = np.zeros_like(s_d, dtype=float)

        s[mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
         

        alpha = np.repeat(s, 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[2*l*(m+1) + j*2: 2*l*(m+1) +j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            # 2*l*(m+1) this part is to take from Q by l 
            
            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] # :-j*2 it works but not for 0 

        d[k] = dd[::2] + dd[1::2]
            
            
        if (d[k] <= factor).all():
            print(k, ": ", d[k],"\n")
            count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
Number_l, len(Q), len(df)

In [ ]:
d

#

#

#

#

# Test for Q all possible values ! 

In [ ]:
import pandas as pd

values = list(range(10))  

# Duplicate each value
duplicated_values = [val for val in values for _ in range(2)]


df2 = pd.DataFrame({'dc_flux': duplicated_values})

print(df2)

In [ ]:
# Define the size of each chunk
m=1
chunk_size = 2 * (m + 1)

# Calculate the number of chunks
num_chunks = len(df2[['dc_flux']]) // chunk_size

# Split the DataFrame into chunks and store them in an array
Q = [df2[['dc_flux']].iloc[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]

Q

In [ ]:
for j in range(1, m+1):
    Q.extend([df2[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
    #print([df2[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
############################### importante !!! 
Q



In [ ]:
O = pd.concat(Q)

# Display the resulting DataFrame
print(O)

In [ ]:
m=1
(len(O)) // (2*(m+1))

In [ ]:
m=2
j=2
l=0
#np.tile(Q[(j*2):(j*2+2)], (4 // 2, 1)).ravel()
O[2*l*(m+1) +(j*2) : 2*l*(m+1) +(j*2+2)]

#

#

#

#

# distance dict to dataframe 

In [ ]:
d

In [ ]:
list_of_dicts = []

# Iterate through the data dictionary and create dictionaries for each object ID
for object_id, distances in d.items():
    for distance in distances:
        list_of_dicts.append({'objectId': object_id, 'distance': distance})

# Create DataFrame from the list of dictionaries
pdf = pd.DataFrame(list_of_dicts)

# Display the DataFrame
print(pdf)

In [ ]:
len(df)/2898

In [ ]:
pdf['is_distance_gt_factor'] = pdf['distance'] <= factor

In [ ]:
pdf

# Trash

In [ ]:
df[df['objectId'] =='ZTF18acevrat' ]

In [ ]:

import timeit

def approach1():
    condition = df['source'] == 0
    df['dc_sigflux_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
    df['nr_sigflux_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

def approach2():
    df['dc_sigflux_weight'] = 1 / (df['dc_sigflux'] ** 2)
    df['nr_sigflux_weight'] = 1 / (df['nr_sigflux'] ** 2)
    df['dc_sigflux_weight'] = df['dc_sigflux_weight'] * (df['source'] != 0)
    df['nr_sigflux_weight'] = df['nr_sigflux_weight'] * (df['source'] != 0)

time_approach2 = timeit.timeit(approach2, number=10)
time_approach1 = timeit.timeit(approach1, number=10)

print("1 execution time:", time_approach1)
print("2 execution time:", time_approach2)


In [ ]:
m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?

T = {unique_ids.index(k): df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:1]}
d = {unique_ids.index(k): np.zeros(int(len(T[k])/2), dtype=np.float64) for k in unique_ids[0:1]}

for j in range(1,m+1): 
    #h = Q.values     
    
    
            for filt in np.unique(T[k]['fid']):
                mask = T[k]['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[k][mask]['dc_flux'].values
                w = T[k][mask]['dc_sigflux_weight'].values

                s_n = (f[-1+j:]*h[j-1] * w[-1+j:])
                s_d = (h[j-1]**2 * w[-1+j:])

                alpha = s_n / s_d
                #print("alpha", alpha)
                
                d[k] += ((f[-1+j:] - alpha * h[j-1])**2) * w[-1+j:]

                #break
            
if (d[k] <= factor).any():
        #print(d)
        count +=1
        #print(count, k)
        
print(count)

In [ ]:
import time 
start_time = time.time()



m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?
#print(Q)

#T = {k: df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:100]}
d0 = {k: 0 for k in unique_ids[0:10]}
#print(d)

count = 0

for k in unique_ids[0:10]:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_sigflux_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn 
        for j in range(0,m): 
            
            alpha = 0
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                s_n = (f[j:]*h[j] * w[j:])
                s_d = (h[j]**2 * w[j:])
                s = np.where(s_d == 0, 0, s_n / s_d)

                alpha += s
                #print("alpha", alpha)
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                d0[k] += ((f[j:] - alpha * h[j])**2) * w[j:]
               #print()
               #print(d)

            #break
            
    if (d0[k] <= factor).all():
        print(d0[k],k)
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")
       